In [ ]:
import pandas as pd
#pd.set_option('display.max_colwidth', -1)
import numpy as np
import glob
import os
import re
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from numpy import random
import zipfile as zf

# this houses folder of psytoolkit data, this notebook, and image sequence csv
main_dir = '/home/jovyan/'

# extracts zip file of psytoolkitdata, remember to move the actual folder up one 
files = zf.ZipFile("test_data.zip", 'r')
files.extractall(main_dir)
files.close()

# path to psytoolkitdata
path = main_dir + 'test_data/'
print(path)

# convert all face task tab delimited txt files to csv
for file in glob.iglob(path + 'Face_Task' + '*.txt', recursive=True):
   
    dataframe = pd.read_csv(file, delimiter=' ')
    base, ext = os.path.splitext(file)
    dataframe.to_csv(base + '.csv', index=False)
sub_ids = []

num_subs = 0
num_good_subs = 0
all_slopes = []
all_midpts = []

for filename in glob.iglob(path + 'Face_Task' + '*.csv', recursive=True):
    
    num_subs = num_subs + 1
    
    file = [filename]
    print(file)
    
    # Sigmoid function definition
    def sigmoid(x, x0, k, a, c):
        y = a / (1 + np.exp(-k*(x-x0))) + c
        return y
    imageThreatData = pd.read_csv('Image_Sequence_Online.csv',header=None)
    imageThreatData.columns=["Threat Level","ImagePerson","Noise Level"]
    imageThreatData['Threat Level']=imageThreatData['Threat Level'].astype(float).apply(lambda x: x/10)
    imageNum=1
    imageThreatData.loc[0,'ImageNum'] = 1
    for i in range(len(imageThreatData['ImagePerson'])-1):
        if (imageThreatData.loc[i+1,'Noise Level']) == 35:   
            imageNum=imageNum + 1
        imageThreatData.loc[i+1,'ImageNum'] = imageNum
    responseTimes = pd.DataFrame(columns=file)
    responseTimes.insert(0,'ImageNum',1)
    responseTimes['ImageNum']=range(1,101)
    allAngProbs=pd.DataFrame()
    subjectInflections=pd.DataFrame(columns=['Subject','Inflection Point x','Inflection Point y'])
    f = file[0]
    subjectData = pd.read_csv(f,header=None,names=["Keypress","Programmed Time","Time in Loop","Trial"])
    imageNum=1
    subjectData.loc[:,'ImageNum'] = imageNum
    for i in range(2,len(subjectData['Trial'])-1):
        if ((subjectData.loc[i,'Trial'] == 36)&(subjectData.loc[i+1,'Trial'] == 37))|((subjectData.loc[i,'Trial'] == 72)&(subjectData.loc[i+1,'Trial'] == 1)):
            imageNum = imageNum +1    
        subjectData.loc[i+1,'ImageNum'] = imageNum
    #DELETE DUPLICATE PRESSES
    subjectData = subjectData.drop_duplicates(subset=['Trial','ImageNum','Keypress','Programmed Time'],keep='first')
    #OBTAIN THREAT LEVELS FOR EACH IMAGE
    subjectData = pd.merge(subjectData,imageThreatData[['Threat Level','ImageNum']].drop_duplicates(subset=['Threat Level','ImageNum']),left_on='ImageNum',right_on='ImageNum')
    #RESPONSE TIMES AND THEIR STATISTICS    
    twoSwitchesInNextImage=False
    for imageNum in range(1,101):
        curIm = subjectData[subjectData['ImageNum']==imageNum].reset_index()
        nextIm = subjectData[subjectData['ImageNum']==imageNum+1].reset_index()
        responseTime=0
        # HOW MANY TIMES (NO PRESS -> SOME KEY PRESSED) IN AN IMAGE?
        threeToTwo=False
        threeToOne=False
        for i in range(len(nextIm['Keypress'])-1):
            if ((nextIm.loc[i,'Keypress']==3 and nextIm.loc[i+1,'Keypress']==2)):
                    threeToTwo=True
            if((nextIm.loc[i,'Keypress']==3 and nextIm.loc[i+1,'Keypress']==1)):
                    threeToOne=True
        if threeToTwo==True & threeToOne==True:
            twoSwitchesInNextImage=True
            #print('There have been two different keys pressed in Image # ',imageNum+1)
        # IF NO KEY IS PRESSED: 
        if (not(any(curIm['Keypress'].isin([1]))|any(curIm['Keypress'].isin([2])))):        
            if (any(nextIm.loc[0:2,'Keypress']==1))|(any(nextIm.loc[0:2,'Keypress']==2)) & twoSwitchesInNextImage==True:
                prevImStallTimeSum = curIm['Time in Loop'].sum()-1495
                responseTime=prevImStallTimeSum+nextIm.loc[0,'Programmed Time'].sum()
                #print("Key pressed fast in Image #", imageNum,"because ",imageNum+1," had no keypress.")
                responseTimes.loc[imageNum-1,f]=responseTime  
            else:
                #print("Image # ",imageNum," was a Missed image (no quick keypress in next).")
                responseTimes.loc[imageNum-1,f]="This image was missed."    
        #RECORDING RESPONSE TIMES
        keyAlreadyPressed=False          
        for frame in range(1,len(curIm)-1):
                if (curIm.loc[frame,'Time in Loop']<300):
                    responseTime=responseTime+curIm.loc[frame,'Time in Loop']  
                    # IF KEY PRESSED
                    if ((keyAlreadyPressed==False) & (curIm.loc[frame,'Keypress']==3)&(curIm.loc[frame+1,'Keypress']==1)|(curIm.loc[frame,'Keypress']==3)&(curIm.loc[frame+1,'Keypress']==2)):
                        responseTime=responseTime+curIm.loc[frame+1,'Programmed Time']
                        #print("Registered RT ", responseTime, " of Image ",imageNum," at frame " ,frame+1)
                        responseTimes.loc[imageNum-1,f]=responseTime
                        imageNum=imageNum+1
                        keyAlreadyPressed=True
                else:
                    responseTime=0
    filtered=subjectData.drop_duplicates(subset=['Trial','ImageNum','Keypress'],keep='first')
    filtered=filtered.loc[filtered['Keypress']!=3]
    filtered=filtered.sort_values(by="Threat Level",ascending=True)
    keysByThreat=filtered.groupby(["Threat Level",'Keypress']).size().reset_index().rename(columns={0:'AmtPressed'})
    threatKeys=pd.DataFrame(data={'Threat Level': [.1,.1,.2,.2,.3,.3,.4,.4,.5,.5,.6,.6,.7,.7,.8,.8,.9,.9,1,1], 'Keypress': [1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2]})
    threatKeys2=pd.merge(threatKeys,keysByThreat,how='left').fillna(0)
    # RECORDING PRESSES BY KEY ABND LEVEL
    for i in range(0,len(threatKeys2),2):
        threatKeys2.loc[i,'Tot Press by Lvl']= threatKeys2.loc[i,'AmtPressed']+threatKeys2.loc[i+1,'AmtPressed']
        threatKeys2.loc[i+1,'Tot Press by Lvl']= threatKeys2.loc[i,'AmtPressed']+threatKeys2.loc[i+1,'AmtPressed']
    # COMPUTING AND STORING THE PROBABILITIES 
    angerProbabilities = threatKeys2.loc[threatKeys2['Keypress']==2]
    xdata = angerProbabilities['Threat Level']*10
    ydata =  pd.Series(angerProbabilities['AmtPressed'].div(angerProbabilities['Tot Press by Lvl'], axis=0))
    # PRINTING EACH CURVE FUNCTION's INFO
    
    xc = [0, 1, 2]
    yc = [0, 1, 2]
    r = random.random()
    b = random.random()
    g = random.random()
    color = (r, g, b)
    
    popt, pcov = curve_fit(sigmoid, xdata, ydata,maxfev = int(10e+5),method='dogbox')
    x = np.linspace(1, 10, 5000)
    y = sigmoid(x, *popt)
    
    # checking goodness of fit using r2
    sigfit_x_red = np.linspace(1,10,10)
    sigfit_y_red = sigmoid(sigfit_x_red,*popt)
    ss_res = np.sum((ydata - sigfit_y_red) ** 2)
    ss_tot = np.sum((ydata - np.mean(ydata)) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    print('r2=',r2)
    print('slope=',popt[1])
    print('midpt=',popt[0])
    
    if r2 > 0.9:
        
        num_good_subs = num_good_subs + 1
        all_slopes.append(popt[1])
        all_midpts.append(popt[0])
        print('***This is a good subject!***')
    
    if r2 == r2:
        plt.scatter(xdata, ydata, label=f, s=15,c='black')
        plt.plot(x,y,c=color)
        plt.xticks(np.arange(min(x), max(x)+1, 1.0),fontsize=11)
        plt.yticks(fontsize=11)
        plt.xlim(0.5,10.5)
        plt.ylim(-0.1,1.1)
        plt.xlabel('Threat Level',fontsize=13)
        plt.ylabel('Fraction Identified as Threat',fontsize=13)
        plt.show()


